https://www.kaggle.com/datasets/alphiree/cardiovascular-diseases-risk-prediction-dataset

## Installing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tensorflow import keras
from keras import layers, callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout

## Load The Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CVD_cleaned.csv')
df.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


## Exploring The Dataset

In [ ]:
df.isnull().sum()

General_Health                  0
Checkup                         0
Exercise                        0
Heart_Disease                   0
Skin_Cancer                     0
Other_Cancer                    0
Depression                      0
Diabetes                        0
Arthritis                       0
Sex                             0
Age_Category                    0
Height_(cm)                     0
Weight_(kg)                     0
BMI                             0
Smoking_History                 0
Alcohol_Consumption             0
Fruit_Consumption               0
Green_Vegetables_Consumption    0
FriedPotato_Consumption         0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

In [ ]:
for i in range(len(df.columns)):
  print(df.columns[i])
  print(df[df.columns[i]].unique())
  print('## ' * 50)

General_Health
['Poor' 'Very Good' 'Good' 'Fair' 'Excellent']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Checkup
['Within the past 2 years' 'Within the past year' '5 or more years ago'
 'Within the past 5 years' 'Never']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Exercise
['No' 'Yes']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Heart_Disease
['No' 'Yes']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Skin_Cancer
['No' 'Yes']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 

In [ ]:
## Creating numerical and categorical columns
numerical = df.select_dtypes(include=['float64']).columns.sort_values()
categorical = df.select_dtypes(include=['object']).columns.sort_values()

## EDA

In [ ]:
print(f'There are {len(categorical)} Categorical variables')
print(f'There are {len(numerical)} Numerical variables')

There are 12 Categorical variables
There are 7 Numerical variables


In [ ]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

In [ ]:
categorical

Index(['Age_Category', 'Arthritis', 'Checkup', 'Depression', 'Diabetes',
       'Exercise', 'General_Health', 'Heart_Disease', 'Other_Cancer', 'Sex',
       'Skin_Cancer', 'Smoking_History'],
      dtype='object')

In [ ]:
numerical

Index(['Alcohol_Consumption', 'BMI', 'FriedPotato_Consumption',
       'Fruit_Consumption', 'Green_Vegetables_Consumption', 'Height_(cm)',
       'Weight_(kg)'],
      dtype='object')

In [ ]:
label_encoder = preprocessing.LabelEncoder()

for i in (df[categorical]):
  df[i] = label_encoder.fit_transform(df[i])

df.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,3,2,0,0,0,0,0,0,1,0,10,150.0,32.66,14.54,1,0.0,30.0,16.0,12.0
1,4,4,0,1,0,0,0,2,0,0,10,165.0,77.11,28.29,0,0.0,30.0,0.0,4.0
2,4,4,1,0,0,0,0,2,0,0,8,163.0,88.45,33.47,0,4.0,12.0,3.0,16.0
3,3,4,1,1,0,0,0,2,0,1,11,180.0,93.44,28.73,0,0.0,30.0,30.0,8.0
4,2,4,0,0,0,0,0,0,0,1,12,191.0,88.45,24.37,1,0.0,8.0,4.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  int64  
 1   Checkup                       308854 non-null  int64  
 2   Exercise                      308854 non-null  int64  
 3   Heart_Disease                 308854 non-null  int64  
 4   Skin_Cancer                   308854 non-null  int64  
 5   Other_Cancer                  308854 non-null  int64  
 6   Depression                    308854 non-null  int64  
 7   Diabetes                      308854 non-null  int64  
 8   Arthritis                     308854 non-null  int64  
 9   Sex                           308854 non-null  int64  
 10  Age_Category                  308854 non-null  int64  
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

## Finding the Highly Co-related columns

In [ ]:
co_mat = df.corr()['Heart_Disease']
co_mat

General_Health                 -0.020645
Checkup                         0.083480
Exercise                       -0.096347
Heart_Disease                   1.000000
Skin_Cancer                     0.090848
Other_Cancer                    0.092387
Depression                      0.032526
Diabetes                        0.166241
Arthritis                       0.153913
Sex                             0.072595
Age_Category                    0.229011
Height_(cm)                     0.015780
Weight_(kg)                     0.045875
BMI                             0.042666
Smoking_History                 0.107797
Alcohol_Consumption            -0.036569
Fruit_Consumption              -0.020055
Green_Vegetables_Consumption   -0.024043
FriedPotato_Consumption        -0.009227
Name: Heart_Disease, dtype: float64

## SPlitting the Train & Test Data

In [ ]:
x = df.drop('Heart_Disease', axis=1)
y = df['Heart_Disease']

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
len(xtrain), len(xtest), len(ytrain), len(ytest)

(231640, 77214, 231640, 77214)

# SVC

In [ ]:
# svm_classifier = SVC(kernel='linear', random_state=42)

# svm_classifier.fit(xtrain, ytrain)

In [ ]:
# ypred = svm_classifier.predict(xtest)
# acc = accuracy_score(ytest, ypred)
# acc

## ANN

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True)

# Initialising the ANN
model = Sequential()

# layers
model.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = 18))
model.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
history = model.fit(xtrain, ytrain, batch_size = 25, epochs = 80, callbacks=[early_stopping], validation_split=0.25)

Epoch 1/80
6950/6950 [==============================] - 28s 4ms/step - loss: 0.2518 - accuracy: 0.9192 - val_loss: 0.2533 - val_accuracy: 0.9193
Epoch 2/80
6950/6950 [==============================] - 31s 4ms/step - loss: 0.2373 - accuracy: 0.9193 - val_loss: 0.2371 - val_accuracy: 0.9193
Epoch 3/80
6950/6950 [==============================] - 28s 4ms/step - loss: 0.2367 - accuracy: 0.9193 - val_loss: 0.2550 - val_accuracy: 0.9193
Epoch 4/80
6950/6950 [==============================] - 28s 4ms/step - loss: 0.2359 - accuracy: 0.9193 - val_loss: 0.2375 - val_accuracy: 0.9193
Epoch 5/80
6950/6950 [==============================] - 25s 4ms/step - loss: 0.2349 - accuracy: 0.9193 - val_loss: 0.2378 - val_accuracy: 0.9193
Epoch 6/80
6950/6950 [==============================] - 27s 4ms/step - loss: 0.2351 - accuracy: 0.9193 - val_loss: 0.2368 - val_accuracy: 0.9193
Epoch 7/80
6950/6950 [==============================] - 28s 4ms/step - loss: 0.2345 - accuracy: 0.9193 - val_loss: 0.2344 - val_ac

## Result

In [ ]:
# Predict on training and test sets
train_preds = model.predict(xtrain)
train_pred = (train_preds > 0.5).astype(int)
test_preds = model.predict(xtest)
test_pred = (test_preds > 0.5).astype(int)

# Calculate the accuracy scores
train_accuracy = accuracy_score(ytrain, train_pred)
test_accuracy = accuracy_score(ytest, test_pred)

train_accuracy, test_accuracy

2413/2413 [==============================] - 5s 2ms/step


(0.9193274045933345, 0.9186157950630714)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate classification report
classification_rep = classification_report(ytest, test_pred)

# Generate confusion matrix
confusion_mat = confusion_matrix(ytest, test_pred)


print(classification_rep)
print("\nConfusion Matrix : \n", confusion_mat)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96     70930
           1       0.00      0.00      0.00      6284

    accuracy                           0.92     77214
   macro avg       0.46      0.50      0.48     77214
weighted avg       0.84      0.92      0.88     77214


Confusion Matrix : 
 [[70930     0]
 [ 6284     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
